In [1]:
# import the libraries we will need
import pandas as pd
import numpy as np
import itertools

(a) A drawback of using confidence is that it ignores Pr(B). Why is this a drawback? Explain
why lift and conviction do not suffer from this drawback.(15 points)

    当Pr(B)过高时，B是否出现与A的关系就不那么强，但是confidence依然很高，因此出现误导，不能说明两者的相关性。但lift和conviction会计算S(B)，相当于考虑到了Pr（B），并且lift和conviction的定义中，提到了A和B需要互相独立这一前提条件。

(b) A measure is symmetrical if measure(A → B) = measure(B → A). Which of the measures presented here are symmetrical? For each measure, please provide either a proof that the measure is symmetrical, or a counterexample that shows the measure is not symmetrical.(15 points)

    Lift is symmetrical.

    Lift: 
    lift(A->B) = conf(A->B)/S(B) = Pr(B|A)/S(B) = S(AB)/S(A)S(B)
    同理lift(B->A) = S(AB)/S(B)S(A) = lift(A->B).

    反例：
    Confidence: 
    假设Pr(A) = 1 且 Pr(B) ≠ 1
    则conf(B->A) = 1
    且conf(A->B) = Pr(B)≠1
    conf(B->A) ≠ conf(A->B).

    Conviction:
    假设S(A) = 1，S(B)  =0.5
    则conf(A->B) = Pr(B|A) = 0.5, conf(B->A) = Pr(A|B) = 1
    conv(A->B) = (1-S(B))/(1-conf(A->B)) = 1
    conv(B->A) = (1-S(A))/(1-conf(B->A)) = 0
    conv(B->A) ≠ conv(A->B).

(c) Perfect implications are rules that hold 100% of the time (or equivalently, the associated conditional probability is 1). A measure is desirable if it reaches its maximum achievable value for all perfect implications. This makes it easy to identify the best rules. Which of the above measures have this property? You may ignore 0/0 but not other infinity cases. Also you may find it easy to explain by an example.(20 points)

    confidence：
    有这个性质。对任意perfect implication，conf都能取到最大值1.

    lift：
    没有这个性质。反例：假如A和B每次都会同时出现，则conf(A->B)=conf(B->A)=1,且S(B)=1，所以lift(A->B)=1

    conv：
    有这个性质。对任意perfect implication,1-conf(A->B)会取到零，conv则取到正无穷，为其最大值。

(d) [20pts]
Identify pairs of items (X, Y ) such that the support of {X, Y } is at least 100 . For all such pairs, compute the confidence scores of the corresponding association rules: X -> Y,Y -> X. Sort the rules in decreasing order of confidence scores and list the top 5 rules in the writeup. Break ties, if any, by lexicographically increasing order on the left hand side of the rule.

In [2]:
dataset=[]
with open("browsing.txt","r") as f: 
    for line in f:
        line = line.strip()
        temp = line.split()
        dataset.append(temp)
        
dataset = list(map(frozenset,dataset))

In [3]:
item1 = set(itertools.chain(*dataset))
C1 = [frozenset([i]) for i in sorted(item1)]

In [4]:
def LK(dataset,CK,support_min):
    support = {}
    for row in dataset:
        for item in CK:
            if item.issubset(row):
                support[item] = support.get(item,0)+1
    return {k : v for k,v in support.items() if v >= support_min}

In [5]:
L1 = LK(dataset,C1,100)

In [6]:
def apriori(LK):
    CK = []
    lenLk = len(LK)
    k = len(LK[0])+1
    for i in range(lenLk):
        for j in range(i+1,lenLk):
            L1 = list(LK[i])[:k-2]
            L2 = list(LK[j])[:k-2]
            L1.sort()
            L2.sort()
            if L1 == L2:
                CK.append(LK[i] | LK[j])
    return CK

In [7]:
C2 = apriori(list(L1.keys()))

In [8]:
L2 = LK(dataset,C2,100)

In [9]:
def rule(item):
    left=[]
    for i in range(1,len(item)):
        left.extend(itertools.combinations(item,i))
    return [(frozenset(r),frozenset(item.difference(r))) for r in left]

In [10]:
def conf_result(supportData):
    rules=[]
    for item in supportData:
        if len(item) >= 1:
            rules.extend(rule(item))
    result=[]
    for left,right in rules:      
        conf = supportData[left|right] / supportData[left]
        result.append([left,right,conf])
    result = sorted(result,key=(lambda x:(x[2],x[0])),reverse=True)
    return result

In [11]:
supportData_2={}
for i,j in L2.items():
    supportData_2[i] = j
for i,j in L1.items():
    supportData_2[i] = j

conf2 = conf_result(supportData_2)
conf2[:5]

[[frozenset({'DAI93865'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'GRO85051'}), frozenset({'FRO40251'}), 0.999176276771005],
 [frozenset({'GRO38636'}), frozenset({'FRO40251'}), 0.9906542056074766],
 [frozenset({'ELE12951'}), frozenset({'FRO40251'}), 0.9905660377358491],
 [frozenset({'DAI88079'}), frozenset({'FRO40251'}), 0.9867256637168141]]

(e) [30pts] Identify item triples (X, Y, Z) such that the support of {X, Y, Z} is at least 100. For all such triples, compute the confidence scores of the corresponding association rules: (X, Y ) ⇒ Z,(X, Z) ⇒ Y , (Y, Z) ⇒ X. Sort the rules in decreasing order of confidence scores and list the top 5 rules in the writeup. Order the left-hand-side pair lexicographically and break ties, if any, by lexicographical order of the first then the second item in the pair.

In [12]:
C3 = apriori(list(L2.keys()))

In [13]:
L3 = LK(dataset,C3,100)

In [21]:
supportData_3={}
for i,j in L3.items():
    supportData_3[i] = j
for i,j in L2.items():
    supportData_3[i] = j
for i,j in L1.items():
    supportData_3[i] = j
    
conf3 = conf_result(supportData_3)
conf3[:5]

[[frozenset({'DAI62779', 'DAI88079'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'ELE17451', 'GRO85051'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'GRO73461', 'GRO85051'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'ELE26917', 'GRO85051'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'GRO85051', 'SNA80324'}), frozenset({'FRO40251'}), 1.0]]

In [22]:
[v for v in conf3 if len(v[0]) == 2][:5]

[[frozenset({'DAI62779', 'DAI88079'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'ELE17451', 'GRO85051'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'GRO73461', 'GRO85051'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'ELE26917', 'GRO85051'}), frozenset({'FRO40251'}), 1.0],
 [frozenset({'GRO85051', 'SNA80324'}), frozenset({'FRO40251'}), 1.0]]